### Velocity estimation

Let the flow rate at the mid-point in the flume be described by

$Q(t)=Q_0\sin(\omega t)$

where $Q_0$ is the peak flow rate and $\omega=2\pi/T$ where $T$ is the period for the forcing.

The peak flow rate ($Q_0$) at the mid point of the working section is linearly dependant on the peak motor revolutions per minute ($RPM_0$). 

$Q_0=\beta \frac{\pi RPM_0^2}{4\omega}$

The constant of proportionality ($\beta$) between $RPM$ and $U_0$ has a nonlinear dependance on the flow depth ($h$) and the flow radial velocity ($\omega=2\pi/T$), where T is the period of the forcing. There is a the period dependant attenution of horizontal velocity due to the oscillatory boundary layer.


In [1]:
import numpy as np
import scipy.io
import pandas as pd

At present the analytical expression for $\beta=f(h,\omega)$ has not been determined however a number of values for $\beta$ have been derived experimentally for some combinations of $h$ and $T$.

In [2]:
beta_df=pd.read_csv('beta_data.csv',index_col='sample')
beta_df

,h,T,beta
sample,,,
0,0.023,133.0,1.449026e+07
1,0.023,240.0,2.065100e+07
2,0.033,67.0,9.256365e+06
3,0.033,83.0,NaN
4,0.033,87.0,9.510436e+06
5,0.033,120.0,1.238600e+07
6,0.033,240.0,1.551260e+07
7,0.043,67.0,1.010860e+07
8,0.043,110.0,1.067022e+07


This allows us to approximate the $RPM_0$ required to achieve a desired $U_0=Q_0/(hw)$ where $w$ is the width of the working section

Rearranging and substituting for $U_0$ gives

$RPM_0=\sqrt{\frac{\beta~U_0~4\omega~h~w}{\pi}}$


Finally we need to allow for the programmed conversion factor of motor RPM to control system voltage (on a scale of 600RPM = 10V), which is the actual required input

Selecting some input parameters for $U_0$, $h$, and $T$ 

In [3]:
w = 1.83 # width of flume m
h = 0.043 # m
T = 110.0 # s
U0 = 0.05 # m/s
#
thisSample = beta_df[(beta_df['h']==h) & (beta_df['T']==T)]
thisSample

,h,T,beta
sample,,,
8,0.043,110.0,1.067022e+07


In [4]:
def V0(U0,T,h,beta):
    w = 1.83 # hard coded as this wont change often!
    om = 2*np.pi / T
    rpm=np.sqrt((U0*4.*om*h*w*beta)/(np.pi))
    v = rpm/600*10
    return v

print('V0=%.2f' % V0(U0,T,h,thisSample['beta']))

V0=0.92


The more challenging task is estimating for combinations of $T$, and $h$ that have not been measured. One approach is bivariate interpolation using a linear or polynomial spline.

In [5]:
import scipy.interpolate as si 

In [6]:
# Construct a bi-linear interpolation object
fInt = si.LinearNDInterpolator(beta_df[['h','T']],beta_df['beta'])
beta_est = fInt(h,T)
print('beta_est=%g' % beta_est)

beta_est=1.06702e+07


In [7]:
U0=0.07
h=0.04
T=120.
print('V0=%.2f' % V0(U0,T,h,beta_est))

V0=1.01


Incorporating the interpolation into the function

In [8]:
def V0I(U0,T,h):
    w = 1.83 # hard coded as this wont change often!
    
    beta_est = fInt(h,T)
    
    om = 2*np.pi / T
    rpm=np.sqrt((U0*4.*om*h*w*beta_est)/(np.pi))
    v = rpm/600*10
    return v

In [9]:
U0=0.07
h=0.03
T=120.
print('V0=%.2f' % V0I(U0,T,h))

V0=0.95
